In [22]:
amwell_company_context = "Amwell digitally empowers your healthcare ambitions. Our comprehensive, future-ready Converge™ platform enables hybrid care delivery at scale, anytime and any place—a care experience so seamless that it becomes simply healthcare.Transform siloed telehealth transactions into integrated, informed experiences. Select individual or multiple solutions that work together to power care through a single platform and improve healthcare for all.We connect providers, payers and innovators, working alongside them to create an ecosystem of care that spans across in-person, virtual and automated care.It starts with our future-ready platform—a convergence of technologies, services and devices that enables care delivery at scale, anytime and any place.Our “One Team” focus to digitally transform healthcare delivery and access starts at the top. Our co-founders are pioneers of virtual healthcare and experienced entrepreneurs with a proven track record of successfully founding, growing and leading multiple companies. Our senior leadership team consists of some of the brightest minds in the industry, bringing together an extensive operational experience in healthcare, technology and services.We serve 115 healthsystems, 50 health partners, with 3K active providers and 100M+ active members. We have visit success rate of 98%, 99.9% uptime on the platform and 90% provider patient thumbs up rate."

In [24]:
print(amwell_company_context)

Amwell digitally empowers your healthcare ambitions. Our comprehensive, future-ready Converge™ platform enables hybrid care delivery at scale, anytime and any place—a care experience so seamless that it becomes simply healthcare.Transform siloed telehealth transactions into integrated, informed experiences. Select individual or multiple solutions that work together to power care through a single platform and improve healthcare for all.We connect providers, payers and innovators, working alongside them to create an ecosystem of care that spans across in-person, virtual and automated care.It starts with our future-ready platform—a convergence of technologies, services and devices that enables care delivery at scale, anytime and any place.Our “One Team” focus to digitally transform healthcare delivery and access starts at the top. Our co-founders are pioneers of virtual healthcare and experienced entrepreneurs with a proven track record of successfully founding, growing and leading multip

In [28]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch

In [30]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [32]:
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [70]:
def faq_bot(question):
    context = amwell_company_context
    input_ids = tokenizer.encode( question, context)
    tokens = tokenizer.convert_ids_to_tokens( input_ids)
    sep_id_index= input_ids.index(tokenizer.sep_token_id) # Find the SEP index which sepperates the question from the context in the encoding
    # Create segment embeddings by finding our seperator tokens from our list of input ids)
    # Calculate the number of tokens in segment a and b.
    num_seg_a = sep_id_index+1 #: Number of tokens in the first segment (question +
    num_seg_b =len(input_ids)- num_seg_a # : Number of tokens in the second segment (context).
    # create list of segment ids with 0s for each token indicating segment 1 and 1s for each token indicating segment 2
    segement_ids = [0]*num_seg_a + [1]*num_seg_b
    # feed all this into our model, convert our input ids and token type ids into a torch tensor
    output = model(torch.tensor([input_ids]), token_type_ids = torch.tensor([segement_ids])) # pass the input ids and sewgment ids into a Bert model
    answer_start = torch.argmax(output.start_logits) # Find the start and end position of the answer in the sequence
    answer_end = torch.argmax(output.end_logits)  
    if answer_end >= answer_start:
        answer = ' '.join(tokens[answer_start:answer_end+1])
    else: 
        print(" I dont know the answer to this question, can you ask another one?")
    # Remove any unnecessary ## that may be there from processing with the corrected answer    
    corrected_answer = ''
    for word in answer.split():
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' '+ word
            
    return corrected_answer

In [72]:
faq_bot("How many healthsystems does Amwell serve")

' 115'

In [74]:
faq_bot("What is Converge")

' our comprehensive , future - ready converge™ platform enables hybrid care delivery at scale , anytime and any place'

In [ ]:
# This would be really powerful if we can bind it with a land ghain framework to bring in custom data.